**KAIST, Fall 2020**   
**[CoE202(A)] Fundamentals of Artificial Intelligence**

**Final Project: Cancer Classifier**

**Instructor: Prof. Young-Gyu Yoon**


**1. Setup.**

In [ ]:
##### External Libraries
import os
import time
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import datetime
import math
import sys

!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Asia/Seoul /etc/localtime

##### Mount drive
from google.colab import drive
drive.mount("/content/drive")

##### Own libraries
filepath = "/content/drive/My Drive/CoE202/proj/src"
sys.path.append(filepath)

from util import calc_accuracy, cancer_dataset, validation_sampler
from model_baseline import cancer_classifier, Trainer
from visualize import visualizer_firstlayer, visualizer

##### Load data
!cp -r "{filepath}/train.zip" .
!unzip -q train.zip
!cp "{filepath}/train_label.csv" .

##### define training parameters
# global parameters
batch_sz = 32 # batch size
training_p = 0.90 # % of the data for training.
experiment = "results"  # Save folder of model
train_ep = 10 # number of times the whole process is repeated, including data load
load_model = True # true: load best pre-trained model, false: train from scratch
visualize = False # true: visualize results, false otherwise

# Trainer-specific parameters
population_size = 30
fitness_eval = 1000 # max allowed fitness evaluations
offsprings = 0.3 # percentage of parents to be replaced on each generation
k = 0.4 # percentage of parents to be sampled for tournament selection
mut = 0.1 # mutation probability of a NN layer
max_depth = 15 # max allowed NN depth
epochs = 5
timeout = 60 * 10 # timeout (in minutes)
loss_criterion = nn.CrossEntropyLoss().cuda()
acc_criterion = calc_accuracy

##### Setup the training environment
n_train = math.floor(training_p*4096) 
n_val = 4096 - math.floor(training_p*4096)
print(f"\nTrain set size: {n_train}, Validation set size: {n_val}\n")

if not os.path.exists(f"{filepath}/{experiment}"):
    os.mkdir(f"{filepath}/{experiment}")
if not os.path.exists(f"{filepath}/{experiment}/visualize"):
    os.mkdir(f"{filepath}/{experiment}/visualize")

##### Print some sample images
def sample_print(train_loader):
  (sample_batch, sample_label) = next(iter(train_loader))
  fig = plt.figure(figsize=(12, 12))
  for i in range(4):
    for j in range(4):
      fig.add_subplot(4, 4, (i * 4 + j + 1))
      plt.imshow(sample_batch[i*4+j].permute(1, 2, 0))
      plt.title(f"label : {sample_label[i*4+j]}")
  plt.suptitle("Sample images")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
replace train/1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N

Train set size: 3686, Validation set size: 410

['train/2401.jpg', 'train/2154.jpg', 'train/1429.jpg'] 
       # Id  Values
2400  2401       0
2153  2154       1
1428  1429       1
['train/3206.jpg', 'train/320.jpg', 'train/3222.jpg'] 
       # Id  Values
3205  3206       0
319    320       0
3221  3222       0
<class 'torch.Tensor'>
torch.Size([32, 3, 256, 256])


**2. Training.**

In [ ]:
for _ in range(train_ep):
  ##### dataset creation
  train_idx, valid_idx = validation_sampler(root=f"train", length_list=[n_train, n_val])
  train_dataset = cancer_dataset(root=f"train", csv=f"train_label.csv",
                                train_test="train",
                                idx=train_idx,
                                transform=transforms.Compose([
                                transforms.ToTensor()
                              ]))
  valid_dataset = cancer_dataset(root=f"train", csv=f"train_label.csv",
                                train_test="valid",
                                idx=valid_idx,
                                transform=transforms.Compose([
                                transforms.ToTensor()
                              ]))
  train_loader = torch.utils.data.DataLoader(train_dataset,
                                            batch_size=batch_sz,
                                            shuffle=True,
                                            drop_last=True
                                            )
  valid_loader = torch.utils.data.DataLoader(valid_dataset,
                                            batch_size=batch_sz,
                                            shuffle=False,
                                            drop_last=True
                                            )
  #sample_print(train_loader)

  ##### Train
  trainer = Trainer(population_size, fitness_eval, offsprings, k, mut, max_depth, epochs, 
                    timeout, train_loader, valid_loader, loss_criterion, acc_criterion)

  if load_model:  # Load saved model
    model = cancer_classifier(f"{filepath}/{experiment}/model_state_dict.pt")
    trainer.train_classifier(model)
  else:           # Train from scratch
    model = trainer.train()

  ##### Store results
  model.save(f"{filepath}/{experiment}", "model_state_dict")
  print("Successfully saved.")

  ##### Print results
  print("Trained model:")
  print(model)
  print(f"Accuracy: {model.accuracy}")

**3. Visualization.**

In [ ]:
if visualize:
  print("\nAfter Training!")
  print("Visualizing overall layers (after training)")
  visualizer(model, suptitle="", grayscale=True, n_kernel_coefficient=256, skip_1x1=False,
              plt_save=f"{filepath}/{experiment}/visualize/all_layer_after.png", plt_show=True)
  print("Visualizing only the first layer (after training)")
  visualizer_firstlayer(model, skip_1x1=False, grayscale=True, memo="First layer",
                        plt_save=f"{filepath}/{experiment}/visualize/1st_layer_after.png",
                        plt_show=True)